In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import bioframe 
import pandas as pd
import numpy as np 
import os
import json
from io import StringIO
import random

import pysam
import h5py

from Bio import motifs
from Bio import pairwise2
from Bio.Seq import Seq

In [2]:
import sys

sys.path.insert(0, "/home1/smaruj/akita_utils/")

# from akita_utils import *
import akita_utils

2022-07-20 17:33:02.815563: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /spack/apps/linux-centos7-x86_64/gcc-8.3.0/python-3.9.2-uvcroioc4witkp6qf7mbebof5ix4wlb6/lib:/spack/apps/linux-centos7-x86_64/gcc-8.3.0/pmix-3.1.3-3sm6emyqaxapunh7rwbjvtaqoqe2e5z3/lib:/spack/apps/linux-centos7-x86_64/gcc-8.3.0/openmpi-4.0.2-ipm3dnvlbtxawpi4ifz7jma6jgr7mexq/lib:/spack/apps/linux-centos7-x86_64/gcc-8.3.0/openblas-0.3.8-2no6mfziiclwxb7lstxoos335gnhjpes/lib:/spack/apps/gcc/8.3.0/lib64::/home1/smaruj/software/GSL/lib:/home1/smaruj/software/HTSLIB/lib
2022-07-20 17:33:02.815600: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
genome_open = pysam.Fastafile("/project/fudenber_735/genomes/mm10/mm10.fa")

In [4]:
nr_strong = 2
nr_weak = 2

In [5]:
seq_coords_df = akita_utils.prepare_insertion_tsv(
    h5_dirs = '/project/fudenber_735/tensorflow_models/akita/v2/analysis/permute_boundaries_motifs_ctcf_mm10_model*/scd.h5',
    score_key = 'SCD',
    pad_flank = 0, #how much flanking sequence around the sites to include
    weak_thresh_pct = 1, # don't use sites weaker than this, might be artifacts
    weak_num = nr_weak ,
    strong_thresh_pct = 99, # don't use sites weaker than this, might be artifacts
    strong_num = nr_strong ,
    save_tsv=None, # optional filename to save a tsv
)

29 duplicates removed for  /project/fudenber_735/tensorflow_models/akita/v2/analysis/permute_boundaries_motifs_ctcf_mm10_model7/scd.h5
29 duplicates removed for  /project/fudenber_735/tensorflow_models/akita/v2/analysis/permute_boundaries_motifs_ctcf_mm10_model1/scd.h5
annotating each site with boundary-wide scores
filtering sites by overlap with rmsk
df prepared


In [6]:
seq_coords_df

,index,chrom,start,end,strand,genomic_SCD
0,0,chr12,35192359,35192378,-,53.656250
1,1,chr8,102781112,102781131,-,53.500000
2,2,chr7,139598311,139598330,+,0.058136
3,3,chr8,45342919,45342938,+,0.058319


In [7]:
def add_orientation(seq_coords_df, 
                    nr_strong,
                    nr_weak,
                    mode="same", 
                    orient_list=[">>" for i in range(len(seq_coords_df))]):
    
    df_len = len(seq_coords_df)
    
    mode_types = ["same", "customize", "weak_strong", "for_each"]
    if mode not in mode_types:
        raise ValueError("Invalid mode. Expected one of: %s" % mode_types)
    
    if mode == "same":
        seq_coords_df["orientation"] = [">>" for i in range(df_len)]
    
    elif mode == "customize":
        if len(orient_list) == df_len:
            seq_coords_df["orientation"] = orient_list
        else:
            raise ValueError("Check the length of the customized list of orientation strings. Expected length: %s" % df_len)
    
    elif mode == "weak_strong":
        
        if len(orient_list) != 2:
            raise ValueError("You should provide a list of two orientation strings: first of weak motifs, second - for strong")
        
        weak_orientation = orient_list[0]
        strong_orientation = orient_list[1]
        
        real_orient_list = [weak_orientation for j in range(nr_weak)] + [strong_orientation for k in range(nr_strong)] 
        
        if len(real_orient_list) == df_len:
            seq_coords_df["orientation"] = real_orient_list
        else:
            raise ValueError("Check the numeber of weak and strong sites given. Expected sum of those two numbers is: %s" % df_len)
    
    elif mode == "for_each":
        rep_unit = seq_coords_df
        orientation_ls = []
        
        for o in range(len(orient_list)):
            orientation = orient_list[o]
            orientation_ls = orientation_ls + [orientation for i in range(df_len)]
            if len(seq_coords_df) != len(orientation_ls):
                seq_coords_df = pd.concat([seq_coords_df, rep_unit], ignore_index=True)
            
        seq_coords_df["orientation"] = orientation_ls
        
    seq_coords_df = seq_coords_df.drop(["index"], axis=1)
    return seq_coords_df
    

In [8]:
add_orientation(seq_coords_df, 
                nr_strong = 2,
                nr_weak = 2,
                mode="for_each",
                orient_list=["<<", ">>", ">>>", "<><><>"])

,chrom,start,end,strand,genomic_SCD,orientation
0,chr12,35192359,35192378,-,53.656250,<<
1,chr8,102781112,102781131,-,53.500000,<<
2,chr7,139598311,139598330,+,0.058136,<<
3,chr8,45342919,45342938,+,0.058319,<<
4,chr12,35192359,35192378,-,53.656250,>>
5,chr8,102781112,102781131,-,53.500000,>>
6,chr7,139598311,139598330,+,0.058136,>>
7,chr8,45342919,45342938,+,0.058319,>>
8,chr12,35192359,35192378,-,53.656250,>>>
9,chr8,102781112,102781131,-,53.500000,>>>


In [ ]:
duap[0].shape
# 1st dim - numer of paddings tested
# 2nd dim - number of sequences
# 3rd dim - number of bINSackground sequences
# 4th dim - number of targets

In [ ]:
duap[1][16].shape

In [ ]:
# seq_coords_df.to_csv("./50-60pads_divergent.csv", index=False)

In [ ]:
def save_h5(seq_coords_df, out_dir, filename, stat, prediction):
    chrom = seq_coords_df.chrom.to_numpy()
    start = seq_coords_df.start.to_numpy().astype(np.int32)
    end = seq_coords_df.end.to_numpy().astype(np.int32)
    strand = seq_coords_df.strand.to_numpy()
    genSCD = seq_coords_df.genomic_SCD.to_numpy().astype(np.float64)
    
    with h5py.File(out_dir + "/" + filename, "w") as hf:
        hf.create_dataset("SCD",  data=prediction[0])
        for key in prediction[1]:
            hf.create_dataset("INS-" + str(key),  data=prediction[1][key])
        hf.create_dataset("Chromosome",  data=chrom.astype('S'))
        hf.create_dataset("Start",  data=start)
        hf.create_dataset("End",  data=end)
        hf.create_dataset("Strand",  data=strand.astype('S'))
        hf.create_dataset("genomic_SCD",  data=genSCD)

In [ ]:
save_h5(seq_coords_df=seq_coords_df, 
        out_dir=".", 
        filename="test.h5", 
        stat=["SCD", "INS-16", "INS-20"], 
        prediction=duap)

In [ ]:
import h5py

with h5py.File('test.h5', 'w') as hf:
    hf.create_dataset("SCD",  data=duap[0])
    for key in duap[1]:
        hf.create_dataset("INS-" + str(key),  data=duap[0])
    hf.create_dataset("Chromosome",  data=chrom.astype('S'))
    hf.create_dataset("Start",  data=start)
    hf.create_dataset("End",  data=end)
    hf.create_dataset("Strand",  data=strand.astype('S'))
    hf.create_dataset("genomic_SCD",  data=genSCD)